In [0]:
pip install mlflow

In [0]:
import os
import sys
import pandas as pd
import numpy as np
from numpy import savetxt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn import datasets
import mlflow
import mlflow.sklearn
import warnings
warnings.filterwarnings("ignore")

In [0]:
data = datasets.fetch_california_housing()
X = pd.DataFrame(data= data.data)
X.columns = [_.lower() for _ in data.feature_names]
y = pd.DataFrame(data.target)
y.columns=['target']
df = pd.concat([X,y], axis=1)
df.head(5)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=42, shuffle=True)

In [0]:
mlflow.set_experiment('/california_housing')

In [0]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [0]:
with mlflow.start_run():
    alpha = 0.35
    l1_ratio = 0.75

    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    (rmse, mae, r2) = eval_metrics(y_test, y_pred)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "model")

    savetxt('predictions.csv', y_pred, delimiter=',')
  
    mlflow.log_artifact("predictions.csv")

In [0]:
with mlflow.start_run():
    mlflow.sklearn.autolog(log_input_examples=True)
    
    alpha = 0.09
    l1_ratio = 0.9

    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    (rmse, mae, r2) = eval_metrics(y_test, y_pred)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    #mlflow.log_param("alpha", alpha)
    #mlflow.log_param("l1_ratio", l1_ratio)
    #mlflow.log_metric("rmse", rmse)
    #mlflow.log_metric("r2", r2)
    #mlflow.log_metric("mae", mae)
    #mlflow.sklearn.log_model(lr, "model")

    savetxt('predictions.csv', y_pred, delimiter=',')
  
    mlflow.log_artifact("predictions.csv")